## Searching for bouts for a day of alsa recording

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2022-10-24 11:11:31,097 root         INFO     Running on txori.ucsd.edu


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb

### Get the file locations for a session (day) of recordings

In [3]:
reload(et)
sess_par = {'bird': 's_b1376_22',
           'sess': '2022-10-23'}

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='alsa')

raw_folder = exp_struct['folders']['alsa']
bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts_ceciestunepipe')

In [4]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1376_22',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23',
 'alsa': '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa',
 'kwik': '/scratch/earneodo/s_b1376_22/alsa/kwik/2022-10-23',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1376_22/2022-10-23/alsa',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1376_22/2022-10-23/alsa',
 'tmp': '/scratch/earneodo/tmp',
 'msort': '/scratch/earneodo/s_b1376_22/alsa/msort/2022-10-23',
 'ksort': '/scratch/earneodo/s_b1376_22/alsa/ksort/2022-10-23/',
 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1376_22/2022-10-23/alsa/'}

### read a session's bouts pandas
- get the location of the file
- load the pandas pickle
- load the search/spectrogram parameters

In [5]:
pd.__version__

'1.4.2'

In [6]:
reload(sb)
hparams, bpd = sb.read_session_bouts(sess_par['bird'], sess_par['sess'], recording_software='alsa', curated=False)

In [7]:
bpd.head(5)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,123860,127325,5945280,6111600,"[1.531360110277605, 15.504246562729664, 12.086...",0.769675,15.504247,True,/mnt/sphere/speech_bci/raw_data/s_b1376_22/202...,3465,"[[123860, 123890], [126670, 126705], [127295, ...",3,"[1, 249, 297, 406, 513, 562, 571, 688]",8,433.125000,"[-2, -2, -3, -2, -2, -2, -1, -1, -2, -2, -2, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
1,284340,287720,13648320,13810560,"[9.121982859860587, 49.58309796674664, 67.9034...",0.716615,67.903419,True,/mnt/sphere/speech_bci/raw_data/s_b1376_22/202...,3380,"[[284340, 284570], [287545, 287720]]",2,"[2, 13, 20, 33, 645, 658, 669]",7,482.857143,"[-2, -3, -4, -3, -2, -2, -3, -2, -4, -3, -2, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
2,917215,920320,44026320,44175360,"[48.51281236977588, 66.83052629466528, 53.9875...",0.503067,66.830526,True,/mnt/sphere/speech_bci/raw_data/s_b1376_22/202...,3105,"[[917215, 917285], [920270, 920320]]",2,"[1, 12, 20, 618]",4,776.250000,"[7, 8, 8, 7, 7, 7, 7, 8, 8, 7, 8, 9, 8, 8, 9, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
3,1590600,1594470,76348800,76534560,"[32.63189559940535, 4.672684963565271, 3.69854...",0.503067,32.631896,True,/mnt/sphere/speech_bci/raw_data/s_b1376_22/202...,3870,"[[1590600, 1590630], [1592620, 1592650], [1594...",3,"[134, 404, 767]",3,1290.000000,"[7, 6, 13, 12, 3, 0, -6, -5, 4, 9, 9, 9, 7, 4,...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
4,428905,432335,20587440,20752080,"[2.058584150709947, 3.4987529274101137, 6.5376...",4.577540,13.866584,True,/mnt/sphere/speech_bci/raw_data/s_b1376_22/202...,3430,"[[428905, 429195], [430555, 430615], [430855, ...",5,"[2, 11, 21, 32, 41, 50, 290, 334, 392, 399, 61...",13,263.846154,"[-2, -5, -3, 5, 13, 19, 17, 15, 11, 4, 0, -6, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True


In [8]:
bpd.index.size

110

###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

## inspect the bouts and curate them

#### visualize one bout

In [9]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [10]:
def viz_one_bout(df: pd.Series, sub_sample=1):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    
    
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

#viz_one_bout(bpd.iloc[1])

## use it in a widget


In [11]:
bpd.sort_values('len_ms', ascending=False, inplace=True)
bpd.reset_index(drop=True, inplace=True)

In [12]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [13]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_call = None
        self.is_confusing = None
                
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        self.audio = None
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        # is this the best place for this?
        self.bouts_pd['is_call'] = False
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_call = widgets.Checkbox(description='calls')
        self.is_confusing = widgets.Checkbox(description='confusing')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Song", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Noise", button_style='danger',
                                                 icon='wrong')
        self.buttons['Call'] = widgets.Button(description="Calls")
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        
        top_box = widgets.HBox([self.buttons['Prev'], self.buttons['Next']])
        bottom_box = widgets.HBox([self.buttons['Uncheck'], self.buttons['Check'], self.buttons['Call']])
        button_box = widgets.VBox([top_box, bottom_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1, step=1, 
                                        description="Bout candidate index")
        
        
        #self.audio = Audio(data=np.zeros(100), rate=self.s_f)
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing, self.is_call]),
                                  widgets.VBox([self.m_pick])]
                                  )
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_call.observe(self.call_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            #print('prev clicked')
            curr_bout.value -= 1
        
        elif button.description == 'Song':
            #print('check clicked')
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            self.bouts_pd.loc[self.bout_id, 'is_call'] = False
            curr_bout.value += 1
        
        elif button.description == 'Noise':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            self.bouts_pd.loc[self.bout_id, 'is_call'] = False
            curr_bout.value += 1
        
        elif button.description == 'Calls':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            self.bouts_pd.loc[self.bout_id, 'is_call'] = True
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
        #self.bouts_pd.loc[self.bout_id, 'is_call'] = False # if it is 
    
    def call_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'is_call'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_call.value = bool(self.bout_series['is_call'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
                
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
        #self.audio = Audio(data=self.x.flatten(), rate=self.s_f)
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bpd)

In [15]:
raise KeyboardInterrupt

KeyboardInterrupt: 

In [16]:
np.where((viz_bout.bouts_pd['bout_check']==True) & (viz_bout.bouts_pd['confusing']==False) & (bpd['len_ms']>10000))[0].size

34

### save it

In [17]:
### get the curated file path
##save to the curated file path
bouts_curated_file_path = os.path.join(bouts_folder, hparams['bout_curated_file'])

os.makedirs(bouts_folder, exist_ok=True, mode=0o777)

viz_bout.bouts_pd.to_pickle(bouts_curated_file_path)
logger.info('saved curated bout pandas to pickle {}'.format(bouts_curated_file_path))

2022-10-24 11:15:20,892 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/s_b1376_22/2022-10-23/alsa/bouts_ceciestunepipe/bout_checked.pickle


### Curated bouts longer than 10 seconds

In [18]:
def sess_bout_summary(bout_pd: pd.DataFrame, ax_dict: dict=None, bouts_type='curated', min_len_ms=10000) -> pd.DataFrame:
    ## make and plot a summary of the bird's bout.
    # get lengths of the bouts
    # get estimate timestamps of bouts
    # plot histogram length of bouts
    # histogram of time of bouts
    bout_sel = (bout_pd['valid']==True) & (bout_pd['len_ms'] > min_len_ms);
    if bouts_type=='curated':
        bout_sel = bout_sel & (bout_pd['bout_check']==True) & (bout_pd['confusing']==False) & (bout_pd['is_call']==False);
    
    # len/time? (when do they sing the longest?)
    logger.info('Number of bouts: {}'.format(bout_pd.loc[bout_sel].index.size))
    logger.info('Length of all bouts (minutes): {}'.format(bout_pd.loc[bout_sel, 'len_ms'].values.sum()/60000))
    
    if ax_dict is None:
        bout_pd.loc[bout_sel].hist(column='len_ms')
    return bout_pd

bpd = sess_bout_summary(bpd)

2022-10-24 11:15:20,910 root         INFO     Number of bouts: 19
2022-10-24 11:15:20,912 root         INFO     Length of all bouts (minutes): 4.44725


In [19]:
viz_bout.bouts_pd['file'][0]

'/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/09-00-01-01.wav'

In [20]:
viz_bout.bouts_pd[viz_bout.bouts_pd['bout_check']==True]['file'].values

array(['/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/09-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/10-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/09-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/09-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/09-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/09-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/09-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/11-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/09-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/10-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/alsa/09-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1376_22/2022-10-23/als

In [21]:
viz_bout.bouts_pd.head(1)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing,is_call
0,1493840,1518830,71704320,72903840,"[3.6473360653916944, 6.550008754854345, 4.9818...",4.57754,89.838394,True,/mnt/sphere/speech_bci/raw_data/s_b1376_22/202...,24990,...,20,"[6, 17, 24, 202, 215, 351, 360, 376, 383, 393,...",86,290.581395,"[6, 7, 6, 7, 6, 6, 7, 7, 6, 7, 7, 5, 6, 6, 7, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,True
